### Find TCs for a list of sites and compute water levels
This might be a useful screening model

In [9]:
import os
import numpy as np
import pandas as pd
import requests

# ============================================================
# USER INPUT: SITES TABLE (EDIT THIS)
# ============================================================
# L_m = effective fetch (m)
# h_m = representative depth (m)
sites = pd.DataFrame([
    dict(site_id="site1", lat=35.21, lon=-75.70, L_m=60e3, h_m=4.0),
    # dict(site_id="site2", lat=..., lon=..., L_m=..., h_m=...),
])

sites = pd.DataFrame( [ dict( site_id = "Pamlico", lat=35.253,   lon=-75.70,  L_m=85e3, h_m=4.0), 
            dict( site_id = "Galveston", lat=39.3966, lon=-94.834, L_m=70e3,  h_m=4.0 ), 
             dict( site_id = "E. Matagorda", lat=28.7039, lon=-95.8407, L_m=70e3,  h_m=4.0 )] )

# ============================================================
# IBTRACS INPUT
# ============================================================
IBTRACS_URL = "https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r01/access/csv/ibtracs.NA.list.v04r01.csv"
IBTRACS_LOCAL = "ibtracs.NA.list.v04r01.csv"

# Which agency-specific fields to use
LAT_COL = "USA_LAT"
LON_COL = "USA_LON"
WIND_COL = "USA_WIND"    # knots
RMW_COL  = "USA_RMW"     # nautical miles in IBTrACS list files
NAME_COL = "NAME"
SID_COL  = "SID"
TIME_COL = "ISO_TIME"

# ============================================================
# CROSSING DETECTION SETTINGS
# ============================================================
# crossing tolerance (km) for final screening
CROSS_TOL_KM = 25.0

# require storm to be tropical/subtropical? (set None to ignore)
# NATURE in IBTrACS often includes: TS, HU, TY, etc. Can be messy across agencies.
NATURE_ALLOW = None  # e.g. {"TS","HU"} or None

# ============================================================
# SIMPLE CYCLONE WIND MODEL SETTINGS
# ============================================================
# Modified Rankine exponent outside RMW
ALPHA_OUT = 0.5

# Reduce gradient/track winds to 10 m? (simple scalar)
V10_FACTOR = 0.90

# Translation wind multiplier (adds some asymmetry)
TRANS_FACTOR = 1.00

# ============================================================
# SETUP MODEL SETTINGS
# ============================================================
rho_air = 1.22
rho_w   = 1025.0
g_accel   = 9.81
Cd      = 1.5e-3  # best-guess; you can make Cd a function of speed if desired

# ============================================================
# HELPERS
# ============================================================
def ensure_file(url, path):
    if os.path.exists(path) and os.path.getsize(path) > 0:
        return path
    print(f"Downloading {url} -> {path}")
    r = requests.get(url, stream=True, timeout=300)
    r.raise_for_status()
    with open(path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)
    return path

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.deg2rad(lat1); lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2); lon2 = np.deg2rad(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2.0 * R * np.arcsin(np.sqrt(a))

def bearing_deg(lat1, lon1, lat2, lon2):
    """Bearing from (lat1,lon1) to (lat2,lon2), deg clockwise from north."""
    phi1 = np.deg2rad(lat1); phi2 = np.deg2rad(lat2)
    dlon = np.deg2rad(lon2 - lon1)
    y = np.sin(dlon) * np.cos(phi2)
    x = np.cos(phi1)*np.sin(phi2) - np.sin(phi1)*np.cos(phi2)*np.cos(dlon)
    brg = (np.rad2deg(np.arctan2(y, x)) + 360.0) % 360.0
    return brg

def uv_from_speed_dir_to(U, dir_to_deg):
    th = np.deg2rad(dir_to_deg % 360.0)
    u = U * np.sin(th)  # east
    v = U * np.cos(th)  # north
    return u, v

def speed_dirfrom_from_uv(u, v):
    U = np.sqrt(u*u + v*v)
    dir_to = (np.rad2deg(np.arctan2(u, v)) + 360.0) % 360.0
    dir_from = (dir_to + 180.0) % 360.0
    return U, dir_from

def modified_rankine(site_dist_km, rmw_km, vmax_ms, alpha_out=0.5):
    """Tangential wind magnitude at radius r using modified Rankine vortex."""
    r = max(site_dist_km, 1e-6)
    R = max(rmw_km, 1e-6)
    if r <= R:
        V = vmax_ms * (r / R)
    else:
        V = vmax_ms * (R / r) ** alpha_out
    return V

def storm_translation_uv(track_times, track_lat, track_lon):
    """Compute translation vector between consecutive track points (m/s) using great-circle distance."""
    # forward differences; last point repeats
    u = np.full(len(track_times), np.nan)
    v = np.full(len(track_times), np.nan)
    for i in range(len(track_times)-1):
        dt = (track_times[i+1] - track_times[i]).total_seconds()
        if not np.isfinite(dt) or dt <= 0:
            continue
        # distance and bearing from point i to i+1
        d_km = haversine_km(track_lat[i], track_lon[i], track_lat[i+1], track_lon[i+1])
        brg = bearing_deg(track_lat[i], track_lon[i], track_lat[i+1], track_lon[i+1])
        # convert to m/s
        spd = (d_km * 1000.0) / dt
        uu, vv = uv_from_speed_dir_to(spd, brg)
        u[i] = uu
        v[i] = vv
    # fill last with previous
    u[-1] = u[-2]
    v[-1] = v[-2]
    return u, v

def setup_eta_m(U10_ms, L_m, h_m, Cd=1.5e-3, g_accel=9.81):
    tau = rho_air * Cd * U10_ms**2
    return (tau * L_m) / (rho_w * g_accel * h_m)

def load_ibtracs_list(path):
    # Usecols: minimal plus wind structure
    usecols = [SID_COL, NAME_COL, "SEASON", "BASIN", TIME_COL, "NATURE",
               LAT_COL, LON_COL, WIND_COL, RMW_COL]
    df = pd.read_csv(path, usecols=lambda c: c in usecols, low_memory=False)
    df[TIME_COL] = pd.to_datetime(df[TIME_COL], utc=True, errors="coerce")
    for c in [LAT_COL, LON_COL, WIND_COL, RMW_COL]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df[NAME_COL] = df[NAME_COL].fillna("").astype(str).str.strip()
    df = df.dropna(subset=[SID_COL, TIME_COL, LAT_COL, LON_COL])
    return df

def find_rmw_crossings_for_site(storm_df, site_lat, site_lon, tol_km=25.0):
    """
    Given a single storm's track dataframe, return list of crossing records
    where distance to site crosses RMW.
    """
    storm_df = storm_df.sort_values(TIME_COL).copy()
    t = storm_df[TIME_COL].to_list()
    lat = storm_df[LAT_COL].values
    lon = storm_df[LON_COL].values

    # RMW in IBTrACS list typically in nautical miles; convert to km
    rmw_nm = storm_df[RMW_COL].values.astype(float)
    rmw_km = rmw_nm * 1.852

    # If no RMW, nothing to do
    if np.all(~np.isfinite(rmw_km)):
        return []

    # Distance from center to site
    d_km = haversine_km(site_lat, site_lon, lat, lon)
    x = d_km - rmw_km  # crossing when x changes sign

    # translation vectors (m/s)
    u_tr, v_tr = storm_translation_uv(storm_df[TIME_COL].dt.to_pydatetime(), lat, lon)

    out = []
    for i in range(len(storm_df)-1):
        if not (np.isfinite(x[i]) and np.isfinite(x[i+1]) and np.isfinite(rmw_km[i]) and np.isfinite(rmw_km[i+1])):
            continue

        # sign change (including exact)
        if x[i] == 0 or x[i+1] == 0 or (x[i] < 0 and x[i+1] > 0) or (x[i] > 0 and x[i+1] < 0):
            # linear interpolation fraction
            denom = (x[i+1] - x[i])
            f = 0.0 if denom == 0 else (-x[i] / denom)
            f = min(max(f, 0.0), 1.0)

            t_cross = storm_df.iloc[i][TIME_COL] + (storm_df.iloc[i+1][TIME_COL] - storm_df.iloc[i][TIME_COL]) * f
            rmw_cross_km = rmw_km[i] + (rmw_km[i+1] - rmw_km[i]) * f
            d_cross_km = d_km[i] + (d_km[i+1] - d_km[i]) * f

            # final tolerance check
            if abs(d_cross_km - rmw_cross_km) > tol_km:
                continue

            # interpolate vmax and translation
            vmax_kt = storm_df.iloc[i][WIND_COL]
            vmax2_kt = storm_df.iloc[i+1][WIND_COL]
            vmax_kt = np.nan if not np.isfinite(vmax_kt) else vmax_kt
            vmax2_kt = np.nan if not np.isfinite(vmax2_kt) else vmax2_kt
            vmax_cross_kt = vmax_kt + (vmax2_kt - vmax_kt) * f if (np.isfinite(vmax_kt) and np.isfinite(vmax2_kt)) else np.nan

            # if vmax missing, you may choose to skip
            if not np.isfinite(vmax_cross_kt):
                continue

            vmax_ms = vmax_cross_kt * 0.514444

            # storm center at crossing (interp)
            latc = lat[i] + (lat[i+1] - lat[i]) * f
            lonc = lon[i] + (lon[i+1] - lon[i]) * f

            # bearing from center to site
            brg_cs = bearing_deg(latc, lonc, site_lat, site_lon)

            # Tangential wind direction "to" for NH cyclones: 90 deg left of radial? (counterclockwise)
            # If radial is from center to site (brg_cs), tangential "to" is brg_cs + 90 (counterclockwise circulation gives flow to the left of radial outward)
            # This convention is a simplification; adjust if you want right/left based on hemisphere/basin.
            dir_to_tan = (brg_cs + 90.0) % 360.0

            # Tangential speed from modified Rankine at r = site distance
            Vtan = modified_rankine(d_cross_km, rmw_cross_km, vmax_ms, alpha_out=ALPHA_OUT)

            # Translation at crossing (interp)
            u_trc = u_tr[i] + (u_tr[i+1] - u_tr[i]) * f if (np.isfinite(u_tr[i]) and np.isfinite(u_tr[i+1])) else u_tr[i]
            v_trc = v_tr[i] + (v_tr[i+1] - v_tr[i]) * f if (np.isfinite(v_tr[i]) and np.isfinite(v_tr[i+1])) else v_tr[i]

            # Combine vectors
            u_tan, v_tan = uv_from_speed_dir_to(Vtan, dir_to_tan)
            u_site = u_tan + TRANS_FACTOR * u_trc
            v_site = v_tan + TRANS_FACTOR * v_trc

            # Reduce to 10 m if desired
            u_site *= V10_FACTOR
            v_site *= V10_FACTOR

            U10, Dfrom = speed_dirfrom_from_uv(u_site, v_site)

            out.append(dict(
                cross_time_utc=t_cross,
                center_lat=latc, center_lon=lonc,
                dist_km=float(d_cross_km),
                rmw_km=float(rmw_cross_km),
                vmax_kt=float(vmax_cross_kt),
                U10_mps=float(U10),
                wind_dir_from_deg=float(Dfrom),
            ))

    return out

# ============================================================
# RUN
# ============================================================
ensure_file(IBTRACS_URL, IBTRACS_LOCAL)
ib = load_ibtracs_list(IBTRACS_LOCAL)

results = []

for _, site in sites.iterrows():
    sid = site["site_id"]
    lat0 = float(site["lat"]); lon0 = float(site["lon"])
    L_m  = float(site["L_m"]);  h_m  = float(site["h_m"])

    # Loop storms by SID; for speed you can pre-filter by basin, season, etc.
    for storm_sid, g_check in ib.groupby(SID_COL, sort=False):
        if NATURE_ALLOW is not None:
            # keep only points where NATURE is in allowed set
            g2 = g_check[g_check["NATURE"].astype(str).isin(NATURE_ALLOW)]
            if len(g2) == 0:
                continue
            g_use = g2
        else:
            g_use = g_check

        # Must have at least some RMW and wind
        if g_use[RMW_COL].notna().sum() < 2 or g_use[WIND_COL].notna().sum() < 2:
            continue

        crossings = find_rmw_crossings_for_site(g_use, lat0, lon0, tol_km=CROSS_TOL_KM)
        if not crossings:
            continue

        storm_name = (g_use[NAME_COL].iloc[0] if len(g_use) else "")
        season = g_use.get("SEASON", pd.Series([np.nan])).iloc[0]
        basin  = g_use.get("BASIN",  pd.Series([""])).iloc[0]

        for cr in crossings:
            eta = setup_eta_m(float(cr["U10_mps"]), L_m=float(L_m), h_m=float(h_m), Cd=float(Cd))
            results.append(dict(
                site_id=sid,
                site_lat=lat0, site_lon=lon0,
                L_m=L_m, h_m=h_m,
                storm_sid=storm_sid,
                storm_name=(storm_name if storm_name else "UNNAMED"),
                season=season,
                basin=basin,
                cross_time_utc=cr["cross_time_utc"],
                U10_mps=float(cr["U10_mps"]),
                wind_dir_from_deg=float(cr["wind_dir_from_deg"]),
                eta_setup_m=float(eta),
                dist_km=float(cr["dist_km"]),
                rmw_km=float(cr["rmw_km"]),
                vmax_kt=float(cr["vmax_kt"]),
            ))

out = pd.DataFrame(results).sort_values(["site_id","cross_time_utc"])
out.to_csv("sites_rmw_crossings_setup_events.csv", index=False)

print("Wrote: sites_rmw_crossings_setup_events.csv")
print(out.head(25).to_string(index=False))
print("\nTotal events:", len(out))


C:\Users\csherwood\AppData\Local\Temp\1\ipykernel_33292\3953066840.py:151: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[TIME_COL] = pd.to_datetime(df[TIME_COL], utc=True, errors="coerce")
C:\Users\csherwood\AppData\Local\Temp\1\ipykernel_33292\3953066840.py:181: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  u_tr, v_tr = storm_translation_uv(storm_df[TIME_COL].dt.to_pydatetime(), lat, lon)
C:\Users\csherwood\AppData\Local\Temp\1\ipykernel_33292\3953066840.py:181: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain

Wrote: sites_rmw_crossings_setup_events.csv
     site_id  site_lat  site_lon     L_m  h_m     storm_sid storm_name season  basin                      cross_time_utc   U10_mps  wind_dir_from_deg  eta_setup_m    dist_km     rmw_km   vmax_kt
E. Matagorda   28.7039  -95.8407 70000.0  4.0 2001157N28265    ALLISON   2001    NaN 2001-06-06 00:37:54.004075960+00:00 22.357142         132.986737     1.591948  92.600000  92.600000 43.526108
E. Matagorda   28.7039  -95.8407 70000.0  4.0 2002249N28266        FAY   2002    NaN 2002-09-07 06:31:55.863992477+00:00 19.132661         243.921514     1.165862  67.448282  67.448282 50.000000
E. Matagorda   28.7039  -95.8407 70000.0  4.0 2002249N28266        FAY   2002    NaN 2002-09-07 10:48:08.475482642+00:00 17.728976          32.462864     1.001068  87.424574  87.424574 37.984305
E. Matagorda   28.7039  -95.8407 70000.0  4.0 2003243N24268      GRACE   2003    NaN 2003-08-31 08:12:04.537840971+00:00 24.346067         205.390658     1.887791 123.854964 12

In [8]:
sites

,site_id,lat,lon,L_m,h_m,side_id
0,Pamlico,35.2530,-75.7000,85000.0,4.0,NaN
1,NaN,39.3966,-94.8340,70000.0,4.0,Galveston
2,NaN,28.7039,-95.8407,70000.0,4.0,E. Matagorda
